## First attempt using a Fully Connected Neural Network (FCNN)

In [7]:
import os
import copy
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from alive_progress import alive_bar

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import sys
sys.path.append('/Users/glucia/Projects/DeepLearning/TrackingML')
from core.data_handler import DataHandler
from core.nn import FCNN
from core.losses import ContrastiveLoss
from utils.terminal_colors import TerminalColors as tc

In [8]:
# Load data (only 2 events)

input_dir = '../../data/train_1'
event_numbers = []
for filename in os.listdir(input_dir):
    if filename.endswith("-cells.csv"):
        event_number = filename.split("-")[0][5:]
        event_numbers.append(event_number)
event_numbers.sort()
print(tc.BOLD+f'Number of events: {len(event_numbers)}'+tc.RESET)

cell_csvs = [input_dir+f'/event{event_number}-cells.csv' for event_number in event_numbers]
hits_csvs = [input_dir+f'/event{event_number}-hits.csv' for event_number in event_numbers]
truth_csvs = [input_dir+f'/event{event_number}-truth.csv' for event_number in event_numbers]

EVENTS_USED = 2
print(tc.GREEN+'Number of events used: '+tc.RESET+tc.BOLD+f'{EVENTS_USED}'+tc.RESET)

data_handler = DataHandler(event_numbers[:2], cell_csvs[:2], hits_csvs[:2], truth_csvs[:2])

N_PARTICLES = 100
train_set = copy.deepcopy(data_handler)
train_set.dataset.query('event_id==1000', inplace=True)
train_set.reduce_to_n_particles(N_PARTICLES)

test_set = copy.deepcopy(data_handler)
test_set.dataset.query('event_id==1001', inplace=True)

mean = train_set.mean()
std = train_set.std()

train_set.feature_scaling(mean, std)
test_set.feature_scaling(mean, std)

print(train_set)

train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=True)
test_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=True)

# get train_loader informations
for batch_idx, (data, target) in enumerate(train_loader):
    print(f'Batch {batch_idx}')
    print(f'  data: {data.shape}')
    print(f'  target: {target.shape}')
    break

Number of events: 1770
Number of events used: 2
DataHandler object.
Dataset:
	- type: <class 'pandas.core.frame.DataFrame'>
	- length: 330825
	- columns: (['x', 'y', 'z', 'value', 'event_id'], ['particle_id'])
x: 
	- shape: torch.Size([330825, 4])
	- type: <class 'torch.Tensor'>
y: 
	- shape: torch.Size([330825])
	- type: <class 'torch.Tensor'>

Batch 0
  data: torch.Size([330825, 4])
  target: torch.Size([330825])


In [9]:
# use gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FCNN(input_dimension=4, output_dimension=3)
criterion = ContrastiveLoss(margin=1.0)

In [10]:
LEARNING_RATE = 0.001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
# Track pairs of hits
def track_pairs(data_loader:DataLoader):
    pairs = []
    labels = []
    for batch_idx, (hits, truth) in enumerate(data_loader):
        for ihit1 in range(len(hits)):
            for ihit2 in range(ihit1+1, len(hits)):
                pairs.append([hits[ihit1], hits[ihit2]])
                labels.append(truth[ihit1] == truth[ihit2])
    return pairs, labels

In [12]:
# Training loop with early stopping
NUM_EPOCHS = 100
PATIENCE = 10
BEST_VAL_LOSS = float('inf')
PATIENCE_COUNTER = 0

train_losses = []

with alive_bar(NUM_EPOCHS) as bar:
    for epoch in range(NUM_EPOCHS):
        if epoch%10 == 0:
            print(f'Epoch {epoch+1}')
        bar()

        model.train()
        train_loss = 0.0
        train_pairs, train_labels = track_pairs(train_loader)

        for (input1, input2), label in zip(train_pairs, train_labels):
            input1, input2, label = input1.unsqueeze(0), input2.unsqueeze(0), torch.tensor([label], dtype=torch.float32)
            optimizer.zero_grad()
            output1 = model(input1)
            output2 = model(input2)
            loss = criterion(output1, output2, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_pairs)

        train_losses.append(train_loss.detach())

        # Validation
        #model.eval()
        #val_loss = 0.0
        #val_pairs, val_labels = track_pairs(train_loader)
        #with torch.no_grad():
        #    for (input1, input2), label in zip(val_pairs, val_labels):
        #        input1, input2, label = input1.unsqueeze(0), input2.unsqueeze(0), torch.tensor([label], dtype=torch.float32)
        #        output1 = model(input1)
        #        output2 = model(input2)
        #        loss = criterion(output1, output2, label)
        #        val_loss += loss.item()
    #
        #val_loss /= len(val_pairs)

        print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {train_loss:.4f}') # , Val Loss: {val_loss:.4f}')

        # Early stopping
        #if val_loss < best_val_loss:
        #    best_val_loss = val_loss
        #    patience_counter = 0
        #    best_model_weights = model.state_dict()
        #else:
        #    patience_counter += 1
        #    if patience_counter >= PATIENCE:
        #        print("Early stopping")
        #        break

    # Load best model weights
    #model.load_state_dict(best_model_weights)
    


on 0: Epoch 1
|▍⚠︎                                      | (!) 1/100 [1%] in 11:44.4 (0.00/s) 


KeyboardInterrupt: 

In [ ]:

plt.plot(train_losses, label = "Train")
#plt.plot(train_accuracies, label = "Train")
#plt.plot(test_accuracies, label = "Test")

plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())

plt.legend()

plt.show()

#print(train_accuracies[-1])
#print(test_accuracies[-1])

In [ ]:
# Generate embedding